In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import datetime
import re

In [2]:
def get_next_page(driver):
    try:
        driver.find_element_by_class_name("ui_button.nav.next.primary").send_keys(Keys.ENTER)
        return True
    except:
        print('마지막 페이지입니다')
        return False

In [3]:
def convert_date(date):
    stay_date = date.split('숙박 날짜: ')
    stay_date = stay_date[1]

    # 20XX년 X월 형태의 str
    stay_date = stay_date.replace('년', '').replace(' ', '').replace('월', '')
    dt = datetime.datetime.strptime(stay_date, '%Y%m')
    dt = dt.date() #20XX-XX-01 형태의 date 객체
    return dt

In [23]:
# convert_date 실행 예시
convert_date('숙박 날짜: 2020년 11월')

datetime.date(2020, 11, 1)

In [4]:
def get_data(driver):
    iteration = True
    df = pd.DataFrame(columns = ['nickname', 'score', 'review', 'stay_date'])
    try:
        time.sleep(20)
        #WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 
        #                                                                    "div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu")))
        # 더보기 클릭
        driver.find_element_by_css_selector("div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu").click()

        time.sleep(5)
        # 숙박 날짜 가져오기
        dates = driver.find_elements_by_class_name("_34Xs-BQm")
        stay_dates = []
        for d in dates:
            stay_date = convert_date(d.text)
            stay_dates.append(stay_date)
            # 날짜 수정 - 코로나 전 1년 반 - 코로나 후 1년 반
            if stay_date < datetime.date(2018,6,1): 
                iteration = False

        # 닉네임 가져오기
        nicknames = driver.find_elements_by_css_selector("div._310S4sqz > div > div._2fxQ4TOx > span > a")
        # 별점 가져오기
        rating_codes = driver.find_elements_by_css_selector("div.oETBfkHU > div._2UEC-y30 > div > span")
        scores = []
        for rc in rating_codes:
            cls_attr = str(rc.get_attribute("class")).split("ui_bubble_rating bubble_")
            score = int(cls_attr[1])
            scores.append(score)

        # 리뷰 가져오기
        reviews = driver.find_elements_by_css_selector("div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu > div.cPQsENeY > q > span")
        
        for i in range(len(nicknames)):
            df.loc[i] = [nicknames[i].text, scores[i], reviews[i].text, stay_dates[i]]
            #print(df.loc[i])

    except ElementClickInterceptedException as e:
        print(e)
    except StaleElementReferenceException as e:
        print(e)

    finally:
        return df, iteration

In [5]:
def hotel_review_crawling(url):
    # 데이터 수집
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(options=options)

    data = pd.DataFrame(columns = ['nickname', 'score', 'review', 'stay_date'])

    #첫페이지 시작
    #데이터 크롤링, df에 추가하기
    #다음페이지 클릭 -> disbaled 나오기 전까지 반복

    driver.get(url)
    driver.implicitly_wait(10)
    iteration = True
    page = 1
    while iteration:
        print(page, '번째 페이지')
        tmp_df, iteration = get_data(driver)
        data = pd.concat([data, tmp_df], axis=0, ignore_index=True)
        if iteration == False:
            break
        iteration = get_next_page(driver) 
        page += 1

    #data.to_csv('./hotel_review_test_sillastay_mapo.csv', encoding="utf-8-sig")
    
    return data

In [6]:
def preprocessing_data(data):
    # 개행문자 제거
    for idx, review in enumerate(data['review']):
        data['review'][idx] = review.replace('\n', ' ')
    # 날짜순으로 내림차순 정렬
    data.sort_values(by='stay_date', ascending=False, 
                     ignore_index=True, inplace=True)
    # 날짜 수정 - 코로나 전 1년 반 - 코로나 후 1년 반
    data = data[data['stay_date'] > datetime.date(2018,5,1)]
    # 특수문자, 이모지 제거
    ###
    return data

In [7]:
# 실제로 실행하는 코드
url = "https://www.tripadvisor.co.kr/Hotel_Review-g297893-d8098377-Reviews-Shilla_Stay_Ulsan-Ulsan.html"

data_sample = hotel_review_crawling(url)
data_sample

1 번째 페이지
2 번째 페이지
3 번째 페이지
4 번째 페이지
5 번째 페이지
6 번째 페이지
7 번째 페이지
8 번째 페이지
9 번째 페이지
10 번째 페이지
11 번째 페이지
12 번째 페이지
13 번째 페이지
14 번째 페이지
15 번째 페이지
16 번째 페이지
17 번째 페이지
18 번째 페이지
19 번째 페이지
20 번째 페이지
21 번째 페이지
22 번째 페이지
23 번째 페이지
24 번째 페이지
25 번째 페이지
26 번째 페이지
27 번째 페이지
28 번째 페이지
29 번째 페이지
30 번째 페이지
31 번째 페이지
32 번째 페이지
33 번째 페이지
34 번째 페이지
35 번째 페이지
36 번째 페이지
37 번째 페이지
38 번째 페이지
39 번째 페이지
40 번째 페이지
41 번째 페이지
42 번째 페이지
43 번째 페이지
44 번째 페이지
45 번째 페이지
46 번째 페이지
47 번째 페이지
48 번째 페이지
49 번째 페이지
50 번째 페이지
51 번째 페이지
52 번째 페이지
53 번째 페이지
54 번째 페이지
55 번째 페이지
56 번째 페이지
57 번째 페이지
58 번째 페이지
59 번째 페이지
60 번째 페이지
61 번째 페이지
62 번째 페이지
63 번째 페이지
64 번째 페이지
65 번째 페이지
66 번째 페이지
67 번째 페이지
68 번째 페이지
69 번째 페이지
70 번째 페이지
71 번째 페이지
72 번째 페이지
73 번째 페이지
74 번째 페이지
75 번째 페이지
76 번째 페이지
77 번째 페이지
78 번째 페이지
79 번째 페이지
80 번째 페이지
81 번째 페이지
82 번째 페이지
83 번째 페이지
84 번째 페이지
85 번째 페이지
86 번째 페이지
87 번째 페이지
88 번째 페이지
89 번째 페이지
90 번째 페이지
91 번째 페이지
92 번째 페이지
93 번째 페이지
94 번째 페이지
95 번째 페이지
96 번째 페이지
97 번째 페이지
98 번째 페이지
99 번째 페이지
100 번째 페이지
101 번째 페

,nickname,score,review,stay_date
0,woo,50,항상 친절하고 깨끗한 신라스테이!\n디자인 분위기 너무 좋고\n회색풍의 전체 분위기...,2021-07-01
1,얍얍,50,직원분이 차분히 안내도와주셔서 편하게 체크인 할 수 있어서 좋았습니다.\n이벤트가로...,2021-06-01
2,Mobile427394,50,울산에 출장을 자주 오는 편입니다.\n그간 묵었던 호텔 중에서는 가장 깔끔한 호텔이...,2021-06-01
3,연재 이,50,처음에는 호텔이 좋아야 얼마나 좋을까 생각을 했습니다.\n하지만 제 큰 오산 인가보...,2021-07-01
4,유미 박,50,"매해 아들 생일에 여행을 합니다\n올해는 울산 고래마을로 장소를 정했고, 공항 도착...",2021-07-01
...,...,...,...,...
625,Maps827992,40,울산 지역에 출장을 가면 항상 값이 싼 숙소만 이용했었은데 이번 3일간의 출장 동안...,2018-10-01
626,b_mi_An,30,"울산 출장갈 때 자주 머물렀던 호텔로, 15번이상 머물렀습니다.\n다른 신라스테이가...",2017-11-01
627,Navigate823753,50,지내기 너무 편해요^^\n시설깨끗하고 직원분들 친절하구요~~아침식사 좋아요~^#^ ...,2018-10-01
628,themews2018,50,서울에서 울산으로 출장올때면 항상 여기 묶는다. 깔금하면서 가성비 좋은 호텔로 여기...,2018-10-01


In [10]:
# 중간에 크롤링이 끊겼을 경우
url_next = "https://www.tripadvisor.co.kr/Hotel_Review-g297893-d8098377-Reviews-or695-Shilla_Stay_Ulsan-Ulsan.html"
data_sample2 = hotel_review_crawling(url_next)

data_sample_concat = pd.concat([data_sample_concat, data_sample2], axis=0, ignore_index=True)
data_sample_concat

1 번째 페이지
2 번째 페이지
3 번째 페이지
4 번째 페이지
5 번째 페이지
6 번째 페이지
7 번째 페이지
8 번째 페이지
9 번째 페이지
10 번째 페이지
11 번째 페이지
12 번째 페이지
13 번째 페이지
14 번째 페이지
15 번째 페이지


,nickname,score,review,stay_date
0,woo,50,항상 친절하고 깨끗한 신라스테이!\n디자인 분위기 너무 좋고\n회색풍의 전체 분위기...,2021-07-01
1,얍얍,50,직원분이 차분히 안내도와주셔서 편하게 체크인 할 수 있어서 좋았습니다.\n이벤트가로...,2021-06-01
2,Mobile427394,50,울산에 출장을 자주 오는 편입니다.\n그간 묵었던 호텔 중에서는 가장 깔끔한 호텔이...,2021-06-01
3,연재 이,50,처음에는 호텔이 좋아야 얼마나 좋을까 생각을 했습니다.\n하지만 제 큰 오산 인가보...,2021-07-01
4,유미 박,50,"매해 아들 생일에 여행을 합니다\n올해는 울산 고래마을로 장소를 정했고, 공항 도착...",2021-07-01
...,...,...,...,...
750,Hwasook P,40,톤다운된 그레이톤과 깔끔한 마룻바닥이 인상적인 곳. 침대에 누워 창밖을 바라봤을때 ...,2018-05-01
751,mayblue2018,50,주말에 가볍게 양산 통도 환타지아에 가족과 함께 놀러 갔다가 근처에서 1박으로 여행...,2018-05-01
752,올 리,50,룸컨디션도 좋습니다~\n직원분들이 친절하고 넘 편해요~^^\n가족여행으로 왔다가 급...,2018-05-01
753,태훈 김,40,룸 컨디션도 좋고\n프론트 데스크 분들도 너무 친절해서 잘 묵고 왔습니다\n시내랑 ...,2018-05-01


In [11]:
data_sample_concat

,nickname,score,review,stay_date
0,woo,50,항상 친절하고 깨끗한 신라스테이!\n디자인 분위기 너무 좋고\n회색풍의 전체 분위기...,2021-07-01
1,얍얍,50,직원분이 차분히 안내도와주셔서 편하게 체크인 할 수 있어서 좋았습니다.\n이벤트가로...,2021-06-01
2,Mobile427394,50,울산에 출장을 자주 오는 편입니다.\n그간 묵었던 호텔 중에서는 가장 깔끔한 호텔이...,2021-06-01
3,연재 이,50,처음에는 호텔이 좋아야 얼마나 좋을까 생각을 했습니다.\n하지만 제 큰 오산 인가보...,2021-07-01
4,유미 박,50,"매해 아들 생일에 여행을 합니다\n올해는 울산 고래마을로 장소를 정했고, 공항 도착...",2021-07-01
...,...,...,...,...
750,Hwasook P,40,톤다운된 그레이톤과 깔끔한 마룻바닥이 인상적인 곳. 침대에 누워 창밖을 바라봤을때 ...,2018-05-01
751,mayblue2018,50,주말에 가볍게 양산 통도 환타지아에 가족과 함께 놀러 갔다가 근처에서 1박으로 여행...,2018-05-01
752,올 리,50,룸컨디션도 좋습니다~\n직원분들이 친절하고 넘 편해요~^^\n가족여행으로 왔다가 급...,2018-05-01
753,태훈 김,40,룸 컨디션도 좋고\n프론트 데스크 분들도 너무 친절해서 잘 묵고 왔습니다\n시내랑 ...,2018-05-01


In [12]:
# 기본 전처리 전 원본 데이터 저장(유실방지)
data_sample_concat.to_csv('./data/hotel_review_shillastay_ulsan_origin.csv', encoding="utf-8-sig")

In [13]:
data = preprocessing_data(data_sample_concat)

In [14]:
data

,nickname,score,review,stay_date
0,woo,50,항상 친절하고 깨끗한 신라스테이! 디자인 분위기 너무 좋고 회색풍의 전체 분위기가 ...,2021-07-01
1,연재 이,50,처음에는 호텔이 좋아야 얼마나 좋을까 생각을 했습니다. 하지만 제 큰 오산 인가보네...,2021-07-01
2,유미 박,50,"매해 아들 생일에 여행을 합니다 올해는 울산 고래마을로 장소를 정했고, 공항 도착하...",2021-07-01
3,SUJIN,50,친절하고 깔끔합니다. 울산 도심에 있어 접근성도 좋고 또 오고싶어요. 신라스테이만의...,2021-07-01
4,라이카X,50,울산에서 호텔은 처음이다. 주차장부터 불편함이 없고 직원들도 친절하다. 작은 것부터...,2021-07-01
...,...,...,...,...
742,sky1004osn,40,편안한 잠자리덕분에 잘쉬다 갑니다. 룸도 깔끔하고 좋았구요. 직원분들도 친절하고좋았...,2018-06-01
743,ym j,40,고민하지 않고 묵게되는 숙소. 쾌적하고 편한곳입니다. 몸과 마음이 편히 쉴 수 있는...,2018-06-01
744,건희 황,40,방도 깔끔하고 야간에 야경도 괜찮습니다. 여름에는 에어컨이 좀 약해 더운 감이 있었...,2018-06-01
745,으니다,50,서울 친구들 숙소로 잡아줬어요! 위치도 좋고 근처에 편의시설도 잘 되어있고 객실 컨...,2018-06-01


In [15]:
# csv 저장을 위한 date to str convert
data['stay_date'] = data['stay_date'].apply(lambda X : X.strftime('%Y-%m-%d'))
# 기본 전처리후 데이터 csv 파일로 저장 
#data.to_csv('./data/hotel_review_shillastay_yeoksam.csv', encoding="utf-8-sig")

<ipython-input-15-7183346d9643>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['stay_date'] = data['stay_date'].apply(lambda X : X.strftime('%Y-%m-%d'))


In [16]:
# 리뷰 내 이모지 제거
data['review'] = data['review'].apply(lambda X : X.encode('euc-kr', 'ignore').decode('euc-kr'))
#data.to_csv('./data/hotel_review_shillastay_yeoksam_del_emoji.csv', encoding="utf-8-sig")

<ipython-input-16-b71e843350f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['review'] = data['review'].apply(lambda X : X.encode('euc-kr', 'ignore').decode('euc-kr'))


In [17]:
# 리뷰 내 특수문자 제거
test_data = data
special = re.compile(r'[^ A-Za-z0-9가-힣+]')
test_data['review'] = [special.sub('', s) for s in test_data['review']]

# 특수문자 제거 후 파일 저장
test_data.to_csv('./data/hotel_review_shillastay_ulsan_final.csv', encoding="utf-8-sig")

<ipython-input-17-36d82654af92>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['review'] = [special.sub('', s) for s in test_data['review']]


In [18]:
test_data

,nickname,score,review,stay_date
0,woo,50,항상 친절하고 깨끗한 신라스테이 디자인 분위기 너무 좋고 회색풍의 전체 분위기가 너...,2021-07-01
1,연재 이,50,처음에는 호텔이 좋아야 얼마나 좋을까 생각을 했습니다 하지만 제 큰 오산 인가보네요...,2021-07-01
2,유미 박,50,매해 아들 생일에 여행을 합니다 올해는 울산 고래마을로 장소를 정했고 공항 도착하자...,2021-07-01
3,SUJIN,50,친절하고 깔끔합니다 울산 도심에 있어 접근성도 좋고 또 오고싶어요 신라스테이만의 무...,2021-07-01
4,라이카X,50,울산에서 호텔은 처음이다 주차장부터 불편함이 없고 직원들도 친절하다 작은 것부터의 ...,2021-07-01
...,...,...,...,...
742,sky1004osn,40,편안한 잠자리덕분에 잘쉬다 갑니다 룸도 깔끔하고 좋았구요 직원분들도 친절하고좋았어요...,2018-06-01
743,ym j,40,고민하지 않고 묵게되는 숙소 쾌적하고 편한곳입니다 몸과 마음이 편히 쉴 수 있는곳 ...,2018-06-01
744,건희 황,40,방도 깔끔하고 야간에 야경도 괜찮습니다 여름에는 에어컨이 좀 약해 더운 감이 있었지...,2018-06-01
745,으니다,50,서울 친구들 숙소로 잡아줬어요 위치도 좋고 근처에 편의시설도 잘 되어있고 객실 컨디...,2018-06-01
